# # Film Eleştirileri ve Bag-of-Words Modellemesi

🎯 Bu zorluğun amacı, metinlerin ***Bag-of-words*** modellemesiyle oynamaktır.

✍️ Aşağıdaki veri setinde, _“olumlu”_ veya _“olumsuz”_ olarak sınıflandırılmış 2000 adet yorum bulunmaktadır.

In [1]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
data.shape

(2000, 2)

## 1. Ön işleme (Preprocessing)

❓ **Soru (Metin Temizleme)** ❓

- Bir cümleyi temizleyecek bir `preprocessing` fonksiyonu yazın ve bunu tüm yorumlara uygulayın. Fonksiyon şunları yapmalıdır:
    - boşlukları kaldırma
    - harfleri küçük harfe çevirme
    - sayıları kaldırma
    - noktalama işaretlerini kaldırma
    - tokenization (kelimelere ayırma)
    - lemmatization (kelime köküne indirgeme)
- Temizlenmiş yorumları `clean_reviews` adlı bir sütunda saklayabilirsiniz.
- Bu aşamada stopword’leri kaldırmayın; nedenini `3. N-gram modelleme` bölümünde açıklayacağız.

In [3]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# Lemmatizer nesnesini başlatma
lemmatizer = WordNetLemmatizer()

def preprocessing(sentence):
    # 1. Boşlukları temizle ve küçük harfe çevir
    sentence = sentence.strip().lower()
    
    # 2. Sayıları kaldır
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # 3. Noktalama işaretlerini kaldır
    sentence = ''.join(char for char in sentence if char not in string.punctuation)
    
    # 4. Tokenization (Kelimelere ayırma)
    tokens = word_tokenize(sentence)
    
    # 5. Lemmatization (Kök bulma) ve Stopwords'ü KORUMA
    # Fiil (verb) çekimlerini de yakalamak için pos='v' parametresi eklenebilir, 
    # ancak varsayılan (isim) haliyle bırakıyoruz.
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Tokenleri tekrar birleştirerek temizlenmiş cümleyi döndür
    return " ".join(lemmatized_tokens)


In [4]:
# Yorumları temizle
# Fonksiyonu tüm veri setine uygulama
data['clean_reviews'] = data['reviews'].apply(preprocessing)

# İlk 5 satırı kontrol ederek temizliği doğrulama
data.head()

,target,reviews,clean_reviews
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couple go to a church party drin...
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard quick movie review damn that...
2,neg,it is movies like these that make a jaded movi...,it is movie like these that make a jaded movie...
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...


❓ **Soru (LabelEncoding)**❓

Hedefinizi LabelEncode ile kodlayın ve `“target_encoded”` adlı bir sütuna kaydedin.

In [6]:
from sklearn.preprocessing import LabelEncoder

# Encoder'ı başlatma
label_encoder = LabelEncoder()

# 'target' sütununu kodlama ve 'target_encoded' sütununa kaydetme
data['target_encoded'] = label_encoder.fit_transform(data['target'])

In [7]:
# Hızlı kontrol
data.head()

,target,reviews,clean_reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couple go to a church party drin...,0
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard quick movie review damn that...,0
2,neg,it is movies like these that make a jaded movi...,it is movie like these that make a jaded movie...,0
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...,0
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...,0


## 2. Bag-of-Words Modellemesi

❓ **Soru (Tek kelimelik sözcüklerle NaiveBayes)** ❓

`cross_validate` kullanarak, metinlerin Bag-of-Words temsilinde eğitilmiş bir Multinomial Naive Bayes modelini puanlayın.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

# 1. Vektörleştirme (Bag-of-Words - Unigram)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['clean_reviews'])
y = data['target_encoded']

# 2. Model Tanımlama
nb_model = MultinomialNB()

# 3. Çapraz Doğrulama (Cross Validation)
# cv=5 ile veriyi 5 parçaya bölüp test ediyoruz
cv_results = cross_validate(nb_model, X, y, cv=5, scoring='accuracy')

# Sonuçları yazdırma
print("Ortalama Doğruluk Puanı (Accuracy):", cv_results['test_score'].mean())

Ortalama Doğruluk Puanı (Accuracy): 0.8165000000000001


## 3. N-gram Modellemesi

👀 Stop kelimeleri kaldırmamanızı istediğimizi hatırlayın. Neden? 

👉 Naive Bayes modelini bigramlarla eğiteceğiz. Bu nedenle, “I do not like coriander” (Kişnişi sevmiyorum) gibi bir cümlede, örneğin bu cümlede olumsuzluğu tespit etmek için “do not” bigramını taramak önemlidir.

❓ **Soru (bigramlarla NaiveBayes)** ❓

`cross_validate` kullanarak, metinlerin 2-gram Bag-of-Words temsilinde eğitilmiş bir Multinomial Naive Bayes modelini puanlayın.

In [10]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.clean_reviews)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    data.target_encoded,
    scoring = "accuracy"
)

round(cv_nb['test_score'].mean(),2)

0.84

🏁 Tebrikler! Artık vektörleştirilmiş metinler üzerinde Naive Bayes modelini nasıl eğiteceğinizi biliyorsunuz.

💾 Not defterinizi `git add/commit/push` yapmayı unutmayın...

